In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import mlflow

### Load latest registered model


In [3]:
from mlflow import MlflowClient

client = MlflowClient()
latest_model_version = client.search_model_versions(
    "name='penguins'",
    max_results=1,
    order_by=["last_updated_timestamp DESC"],
)[0]
print(f"Model version: {latest_model_version.version}")

Model version: 111


In [4]:
payload1 = {
    "island": "Biscoe",
    "culmen_length_mm": 48.6,
    "culmen_depth_mm": 16.0,
    "flipper_length_mm": 230.0,
    "body_mass_g": 5800.0,
    "sex": "MALE",
}

payload2 = [
    {
        "island": "Biscoe",
        "culmen_length_mm": 48.6,
        "culmen_depth_mm": 16.0,
        "flipper_length_mm": 230.0,
        "body_mass_g": 5800.0,
        "sex": "MALE",
    },
    {
        "island": "Torgersen",
        "culmen_length_mm": 44.1,
        "culmen_depth_mm": 18.0,
        "flipper_length_mm": 210.0,
        "body_mass_g": 4000,
    },
    {
        "island": "Torgersen",
        "culmen_length_mm": 39.1,
        "culmen_depth_mm": 18.7,
        "flipper_length_mm": 181.0,
        "body_mass_g": 3750,
    },
]

In [5]:
model = mlflow.pyfunc.load_model(latest_model_version.source)

prediction = model.predict(payload2)
print(prediction)

2024/09/02 14:47:40 WARNING mlflow.pyfunc: The version of CloudPickle that was used to save the model, `CloudPickle 3.0.0`, differs from the version of CloudPickle that is currently running, `CloudPickle 2.2.1`, and may be incompatible


TBD MODEL INPUT       island  culmen_length_mm  culmen_depth_mm  flipper_length_mm  \
0     Biscoe              48.6             16.0              230.0   
1  Torgersen              44.1             18.0              210.0   
2  Torgersen              39.1             18.7              181.0   

   body_mass_g   sex  
0       5800.0  MALE  
1       4000.0   NaN  
2       3750.0   NaN  
TBD Making a prediction
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
TBD Model output [{'prediction': 'Gentoo', 'confidence': 0.9959872364997864}, {'prediction': 'Adelie', 'confidence': 0.5151113271713257}, {'prediction': 'Adelie', 'confidence': 0.7011913657188416}]
[{'prediction': 'Gentoo', 'confidence': 0.9959872364997864}, {'prediction': 'Adelie', 'confidence': 0.5151113271713257}, {'prediction': 'Adelie', 'confidence': 0.7011913657188416}]


### Deploying the model locally

```
$ mlflow models serve -m models:/penguins/42 -p 9876 --no-conda
```


In [1]:
import json

import requests

payload3 = json.dumps(
    {
        "inputs": [
            {
                "island": "Biscoe",
                "culmen_length_mm": 48.6,
                "culmen_depth_mm": 16.0,
                "flipper_length_mm": 230.0,
                "body_mass_g": 5800.0,
                "sex": "MALE",
            },
            {
                "island": "Torgersen",
                "culmen_length_mm": 44.1,
                "culmen_depth_mm": 18.0,
                "flipper_length_mm": 210.0,
                "body_mass_g": 4000,
                "sex": "FEMALE",
            },
            {
                "island": "Torgersen",
                "culmen_length_mm": 39.1,
                "culmen_depth_mm": 18.7,
                "flipper_length_mm": 181.0,
                "body_mass_g": 3750,
                "sex": "MALE",
            },
        ],
    },
)

response = requests.post(
    url="http://54.165.213.187:8080/invocations",
    headers={"Content-Type": "application/json"},
    data=payload3,
)
print(response.json())

{'predictions': [{'prediction': 'Gentoo', 'confidence': 0.9937078952789307}, {'prediction': 'Chinstrap', 'confidence': 0.44345566630363464}, {'prediction': 'Adelie', 'confidence': 0.9986134767532349}]}


In [ ]:
payload4 = json.dumps(
    {
        "inputs": [
            {
                "island": "Torgersen2",
                "culmen_length_mm": 39.1,
                "culmen_depth_mm": 18.7,
                "flipper_length_mm": 181.0,
                "body_mass_g": 3750,
                "sex": "MALE",
            },
        ],
    },
)

response = requests.post(
    url="http://127.0.0.1:9874/invocations",
    headers={"Content-Type": "application/json"},
    data=payload4,
)
print(response.json())

In [14]:
import json

payload4 = [
    {
        "island": "Torgersen",
        "culmen_length_mm": None,
        "culmen_depth_mm": None,
        "flipper_length_mm": None,
        "body_mass_g": None,
        "sex": "None",
    },
]

prediction = model.predict(payload4)
print(prediction)

Handling request...
Here is the config: {'DATABASE': 'penguins'}
Here is the model input:       island  culmen_length_mm  culmen_depth_mm  flipper_length_mm  \
0  Torgersen               NaN              NaN                NaN   

   body_mass_g   sex  
0          NaN  None  
Transforming payload...
Making a prediction using the transformed payload...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step
Processing prediction received from the model...
Data saved successfully!
[{'prediction': 'Adelie', 'confidence': 0.6168336}]


```
mlflow models build-docker --name penguins --model-uri "models:/penguins/87"
mlflow deployments run-local -t sagemaker -m models:/penguins/87 --name penguins -C port=9874
```

```
mlflow sagemaker build-and-push-container
mlflow deployments create -t sagemaker --name penguins -m models:/penguins/1 -C region_name=us-east-1 -C instance-type=ml.m4.xlarge -C instance-count=1
```

Deployment plugins to deploy to cloud providers
https://mlflow.org/docs/latest/plugins.html#deployment-plugins

In [9]:
import sagemaker
from sagemaker.deserializers import JSONDeserializer
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer

sagemaker_session = sagemaker.session.Session()

ENDPOINT = "penguins"

predictor = Predictor(
    endpoint_name=ENDPOINT,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    sagemaker_session=sagemaker_session,
)

In [10]:
sample = {
    "inputs": [
        {
            "island": "Torgersen2",
            "culmen_length_mm": 39.1,
            "culmen_depth_mm": 18.7,
            "flipper_length_mm": 181.0,
            "body_mass_g": 3750,
            "sex": "MALE",
        },
        {
            "island": "Biscoe",
            "culmen_length_mm": 48.6,
            "culmen_depth_mm": 16.0,
            "flipper_length_mm": 230.0,
            "body_mass_g": 5800.0,
            "sex": "MALE",
        },
    ],
}


try:
    response = predictor.predict(sample)
    print(response)
except Exception as e:
    print(e)

{'predictions': [{'prediction': 'Adelie', 'confidence': 0.9968430995941162}, {'prediction': 'Gentoo', 'confidence': 0.9937078952789307}]}
